In [1]:
import os
import math
import numpy as np
import pandas as pd
import pywt
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema, find_peaks
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import cv2
import glob
import scipy.stats as stats

In [2]:
%run Ipynb_importer.py
import Data_extension

importing Jupyter notebook from Data_extension.ipynb


In [5]:
def wavelet_local_maxima(data, scale_in):

    extend_length = 100
    data = Data_extension.Data_extension(data, extend_length)
    
    wname = "mexh"
    coeffs, freqs = pywt.cwt(data, scales=np.arange(1, scale_in), wavelet=wname)
    
    # calculate the local maxima at each time step: for plotting local maxima lines
    # note that the local maxima information is the scale position, not the exact coefficient values
    # note that absolute values of wavelet coefficients should be used
    # note that the method of finding the local maxima is: under certain scale, find all the time correspinding to all the local maxima
    absolute_coeffs = np.absolute(coeffs)
    absolute_coeffs = np.delete(absolute_coeffs, range(extend_length), axis=1)
    absolute_coeffs = np.delete(absolute_coeffs, (np.arange(extend_length) + 1) * (-1), axis=1)
    local_maxima_points = pd.DataFrame()
    for i in range(scale_in - 1):
        current_scale_coeffs = absolute_coeffs[i, :]
        
        #local_maxima_index = argrelextrema(current_scale_coeffs, np.greater, order=2)
        #use find_peaks instead of argrelextrema, round the coef array to avoid some numerical errors
        local_maxima_index = find_peaks(np.round(current_scale_coeffs, 6), width=2)
        local_maxima_index = local_maxima_index[0]
        
        maxima_info = pd.DataFrame(np.zeros((len(local_maxima_index), 2)), columns=['time_index', 'scale_id'])

        for max_i in range(len(local_maxima_index)):
            maxima_info.loc[max_i, 'time_index'] = local_maxima_index[max_i]  # time index position
            maxima_info.loc[max_i, 'scale_id'] = i + 1  # scale id
        local_maxima_points = local_maxima_points.append(maxima_info)
    return local_maxima_points